In [319]:
import numpy as np
import pandas as pd

In [1]:
df = pd.read_csv('train-loan.csv')
df

NameError: name 'pd' is not defined

In [321]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [322]:
df.drop(columns = ['Loan_ID'], inplace = True)
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [323]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   600 non-null    float64
 9   Credit_History     564 non-null    float64
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 57.7+ KB


In [324]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [325]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop(columns = ['Loan_Status']), df['Loan_Status'], random_state = 0)

In [326]:
x_train.nunique()

Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      398
CoapplicantIncome    220
LoanAmount           177
Loan_Amount_Term       9
Credit_History         2
Property_Area          3
dtype: int64

In [327]:
trf1 = ColumnTransformer([
    ('tnf1', SimpleImputer(), [5,6,7,8,9]),
    ('tnf2', SimpleImputer(strategy = 'most_frequent'), [0,1,2,3,4,10])
],remainder = 'passthrough')

In [328]:
trf2 = ColumnTransformer([
    ('tnf3',OneHotEncoder(sparse = False, handle_unknown = 'ignore', drop = 'first'), [0,1,2,3,4,8,9,10,])
], remainder = 'passthrough')

In [329]:
trf3 = ColumnTransformer([
    ('scale', StandardScaler(), slice(0,18))
])

In [330]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

In [331]:
from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier()

In [332]:
from sklearn.linear_model import LogisticRegression

clf3 = LogisticRegression()

In [333]:
from sklearn.tree import DecisionTreeClassifier

clf4 = DecisionTreeClassifier()

In [334]:
from sklearn.pipeline import Pipeline , make_pipeline

Pipe = make_pipeline(trf1, trf2, trf3, clf4)

In [335]:
Pipe.fit(x_train, y_train)

C:\Users\Family PC\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tnf1', SimpleImputer(),
                                                  [5, 6, 7, 8, 9]),
                                                 ('tnf2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [0, 1, 2, 3, 4, 10])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tnf3',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2, 3, 4, 8, 9, 10])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  slice(0, 18, None))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [336]:
pred = Pipe.predict(x_test)

C:\Users\Family PC\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0, 1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [337]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)

0.7207792207792207

In [338]:
import pickle 
pickle.dump(Pipe, open('pipe-loan-prediction.pkl', 'wb'))